In [34]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from statsmodels.formula.api import ols, glm
import statsmodels.api as sm
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
spark = (
 SparkSession.builder.appName("project 1 LR")
 .config("spark.sql.repl.eagerEval.enabled", True)
 .config("spark.executor.memory","4G")
 .config("spark.driver.memory","2G")
 .config("spark.sql.parquet.cacheMetadata", "true")
 .config("spark.sql.session.timeZone", "Etc/UTC")
 .config('spark.driver.maxResultSize', '2048m')
 .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/16 22:06:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/16 22:06:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
data = spark.read.parquet('data/merged_sdf.parquet')

In [57]:
model_feature = ['duration', 'extra', 'Weekend', 'Airport', 'Congestion','tip_amount']
selected_data = data.select(*[col(col_name) for col_name in model_feature])

In [89]:
input_column = ['duration', 'extra', 'Weekend', 'Airport', 'Congestion']
assembler = VectorAssembler(inputCols=input_column, outputCol='features')
assembled_data = assembler.transform(selected_data)

# Split the data into training and testing sets
train_data, test_data = assembled_data.randomSplit([0.7, 0.3], seed=0)

In [96]:
for i in [0.0001, 0.001, 0.01, 0.1, 1]:
    lm = LinearRegression(
        featuresCol='features', 
        labelCol='tip_amount',
        regParam=i, 
        elasticNetParam=0.5
    ).fit(train_data)
    predictions = lm.transform(test_data)
    evaluator = RegressionEvaluator(labelCol='tip_amount', predictionCol='prediction', metricName='rmse')
    rmse = evaluator.evaluate(predictions)
    print('R_sqr : ', lm.summary.r2, "Root Mean Squared Error:", rmse, "regParam: ", i)



R_sqr :  0.4778750701430895 Root Mean Squared Error: 2.8562927058954193 regParam:  0.0001


R_sqr :  0.4778749504244446 Root Mean Squared Error: 2.8562925238277574 regParam:  0.001


R_sqr :  0.4778630101021093 Root Mean Squared Error: 2.8563188327429367 regParam:  0.01


R_sqr :  0.47677297653958617 Root Mean Squared Error: 2.859114102458745 regParam:  0.1


R_sqr :  0.43746036404222644 Root Mean Squared Error: 2.95981032711316 regParam:  1


23/08/17 20:26:31 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 591617 ms exceeds timeout 120000 ms
23/08/17 20:26:31 WARN SparkContext: Killing executors is not supported by current scheduler.
23/08/17 20:26:35 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.B